In [70]:
from sdv.metrics.demos import load_timeseries_demo
from sdv.metrics.timeseries import LSTMDetection, TSFCDetection
import pandas as pd


real_data, synthetic_data, metadata = load_timeseries_demo()

In [74]:
metadata = {'fields': {'subject_id': {'type': 'numerical', 'subtype': 'integer'},
  'dod': {'type': 'datetime'}},
 'entity_columns': ['subject_id'],
 'sequence_index': ''}

In [75]:
metadata

{'fields': {'subject_id': {'type': 'numerical', 'subtype': 'integer'},
  'dod': {'type': 'datetime'}},
 'entity_columns': ['subject_id'],
 'sequence_index': ''}

In [76]:
df = pd.read_csv('real.csv').loc[:, ['store_id', 'date']]

In [79]:
df.columns=['subject_id', 'dod']

In [81]:
TSFCDetection.compute(df, df, metadata)

d:\Environment\anaconda3\envs\msc_project\lib\site-packages\sdmetrics\timeseries\detection.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X = X.append(entity_data)
d:\Environment\anaconda3\envs\msc_project\lib\site-packages\sdmetrics\timeseries\detection.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X = X.append(entity_data)
d:\Environment\anaconda3\envs\msc_project\lib\site-packages\sdmetrics\timeseries\detection.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  X = X.append(entity_data)
d:\Environment\anaconda3\envs\msc_project\lib\site-packages\sdmetrics\timeseries\detection.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat i

ValueError: all input arrays must have the same shape